# Forex
_Roboquant_ supports many type of asset classes. This notebook shows how to back test forex strategies.

In [ ]:
%use roboquant
Welcome()

In [ ]:
Currency.increaseDigits(3) // We want to higher precision in reports for currencies
val feed = AvroFeed.forex() // Use sample data containing 1 year EUR/USD 1 minute bars

## Price Chart
A PriceBarChart replay the stream of events found in a feed and plots the prices it finds for a particular asset.

In [ ]:
val eur_usd = feed.assets.first()

// if useTime = true, non-trading periods will show up as gaps
PriceBarChart(feed, eur_usd, useTime = false) 

## Custom Policy 

In [ ]:
// We wan to allow shorting and since only one asset, much higer stakes per order (80%)
class MyPolicy : FlexPolicy(shorting = true, orderPercentage = 0.8) {

    /**
     * We want to create brack orders
     */
    override fun createOrder(signal: Signal, size: Size, price: Double): Order? {
        val asset = signal.asset
        val trailPercentage = 0.02
        val stopPercentage = 0.005
        return BracketOrder.marketTrailStop(asset, size, price, trailPercentage, stopPercentage) 
    }
    
}

## Price Bar Chart
Roboquant charts aren't designed to be a tool where you manually draw support lines on candlestick charts to make trading decisions. But that doesn't mean candlesticks cannot come in handy. For example to validate if the data in a feed is not corrupted or has some other strange anomalies. This time we don't supply a timeframe, so all prices in the feed will be displayed. 

In the top right part of the plotted graph there is a toolbox that you can use to zoom into a particular area of interest.

In [ ]:
val strategy = EMAStrategy()

// We want to configure lower spread cost than stock trading and a margin account
val broker = SimBroker(pricingEngine = SpreadPricingEngine(1.0), accountModel = MarginAccount())


val roboquant =  Roboquant(strategy, AccountMetric(), broker = broker, policy = MyPolicy())
roboquant.run(feed)

In [ ]:
val account = roboquant.broker.account
account.summary()

In [ ]:
// See the trades we made in a single month
val tf = Timeframe.parse("2022-05", "2022-06")
account.trades.filter { it.time in tf }.summary()

In [ ]:
// Chart the trades we made in a single month and plot them against the prices
PriceBarChart(feed, eur_usd, account.trades, timeframe = tf, useTime = false)

# Metrics

Besides the charts for account related data, we can also plot the metrics that where captured during the run. Where an Account contains only the end state after the run has finished, metrics are captured during each step of the run. This allows for charts that provide more insights into different periods of the run. We used the AccountMetric metric that keeps track of how our account performs during the back-test. We use the account value (= cash + positions), for this sample

In [ ]:
val logger = roboquant.logger

// What metrics are available
logger.metricNames.summary("Metrics Names")

## Metric Chart

In [ ]:
val data = logger.getMetric("account.equity")
MetricChart(data)

## Calender Chart
When evaluating the performance of a strategy, it is sometimes required to find out when the strategy was performing good and bad. A calendar charts allows you to plot a metric value per day, so it is easy to find out which days to further investigate. The slider at top of the chart allows for filtering only those days that are of interest based on their value.

In the example below we plot the equity change to see when the strategy performed good (green) and when bad (red).

In [ ]:
MetricCalendarChart(data.diff())

# Multi-run tests
A good way to back-test your strategy is to run it multiple times over different time-frames. This section shows how charts that come in handy to better understand the results of those type of back-tests. 

## Walk-forward
A commonly used approach in back-testing is the so called walk-forward. The historic data is split in a number of equal periods and then the strategy is run for each period separately. This shows how the strategy would have performed during these different periods and possible different regimes.

In the following code we split the data in periods of 1 month and for each period run it. We then plot again the equity as observed during each run.

In [ ]:
roboquant.reset()

feed.timeframe.split(1.months).forEach { 
    roboquant.run(feed, it)   
}

val data = roboquant.logger.getMetric("account.equity")
MetricChart(data)